In [1]:
import os
import h5py
import shutil
import numpy as np

from utils import phi_shift_and_flipping

MAX_CONSTI = {
    'Jet': 50,
    'Tower': 250,
    'Track': 150,
    'Photon': 2,
}

# Split SR (VBF) and BR (GGF) samples 

In [2]:
def get_dataset_keys(f):
    keys = []
    f.visit(lambda key : keys.append(key) if isinstance(f[key], h5py.Dataset) else None)
    return keys

def create_dataset(f, nevent, MAX_CONSTI):

    f.create_dataset('J1/mask', (nevent, MAX_CONSTI['Jet']), maxshape=(None, MAX_CONSTI['Jet']), dtype='|b1')
    f.create_dataset('J1/pt', (nevent, MAX_CONSTI['Jet']), maxshape=(None, MAX_CONSTI['Jet']), dtype='<f4')
    f.create_dataset('J1/eta', (nevent, MAX_CONSTI['Jet']), maxshape=(None, MAX_CONSTI['Jet']), dtype='<f4')
    f.create_dataset('J1/phi', (nevent, MAX_CONSTI['Jet']), maxshape=(None, MAX_CONSTI['Jet']), dtype='<f4')

    f.create_dataset('J2/mask', (nevent, MAX_CONSTI['Jet']), maxshape=(None, MAX_CONSTI['Jet']), dtype='|b1')
    f.create_dataset('J2/pt', (nevent, MAX_CONSTI['Jet']), maxshape=(None, MAX_CONSTI['Jet']), dtype='<f4')
    f.create_dataset('J2/eta', (nevent, MAX_CONSTI['Jet']), maxshape=(None, MAX_CONSTI['Jet']), dtype='<f4')
    f.create_dataset('J2/phi', (nevent, MAX_CONSTI['Jet']), maxshape=(None, MAX_CONSTI['Jet']), dtype='<f4')

    f.create_dataset('TOWER/mask', (nevent, MAX_CONSTI['Tower']), maxshape=(None, MAX_CONSTI['Tower']), dtype='|b1')
    f.create_dataset('TOWER/pt', (nevent, MAX_CONSTI['Tower']), maxshape=(None, MAX_CONSTI['Tower']), dtype='<f4')
    f.create_dataset('TOWER/eta', (nevent, MAX_CONSTI['Tower']), maxshape=(None, MAX_CONSTI['Tower']), dtype='<f4')
    f.create_dataset('TOWER/phi', (nevent, MAX_CONSTI['Tower']), maxshape=(None, MAX_CONSTI['Tower']), dtype='<f4')

    f.create_dataset('TRACK/mask', (nevent, MAX_CONSTI['Track']), maxshape=(None, MAX_CONSTI['Track']), dtype='|b1')
    f.create_dataset('TRACK/pt', (nevent, MAX_CONSTI['Track']), maxshape=(None, MAX_CONSTI['Track']), dtype='<f4')
    f.create_dataset('TRACK/eta', (nevent, MAX_CONSTI['Track']), maxshape=(None, MAX_CONSTI['Track']), dtype='<f4')
    f.create_dataset('TRACK/phi', (nevent, MAX_CONSTI['Track']), maxshape=(None, MAX_CONSTI['Track']), dtype='<f4')

    f.create_dataset('PHOTON/pt', (nevent, MAX_CONSTI['Photon']), maxshape=(None, MAX_CONSTI['Photon']), dtype='<f4')
    f.create_dataset('PHOTON/eta', (nevent, MAX_CONSTI['Photon']), maxshape=(None, MAX_CONSTI['Photon']), dtype='<f4')
    f.create_dataset('PHOTON/phi', (nevent, MAX_CONSTI['Photon']), maxshape=(None, MAX_CONSTI['Photon']), dtype='<f4')

    f.create_dataset('EVENT/mjj', (nevent,), maxshape=(None,), dtype='<f4')
    f.create_dataset('EVENT/deta', (nevent,), maxshape=(None,), dtype='<f4')
    f.create_dataset('EVENT/type', (nevent,), maxshape=(None,), dtype='<i8')

In [3]:
def split_SR_BR_mjj(h5_path, output_path, mjj_cut=300):

    # read data
    with h5py.File(h5_path, 'r') as f:
        mjj = f['EVENT/mjj'][:]

        SR_range = mjj > mjj_cut
        BR_range = mjj < mjj_cut
        
        root, _ = os.path.splitext(output_path)
        SR_path = f'{root}_in_SR.h5'
        BR_path = f'{root}_in_BR.h5'

        with h5py.File(SR_path, 'w') as f_SR, h5py.File(BR_path, 'w') as f_SB:

            create_dataset(f_SR, SR_range.sum(), MAX_CONSTI)
            create_dataset(f_SB, BR_range.sum(), MAX_CONSTI)

            keys = get_dataset_keys(f_SR)

            for key in keys:
                f_SR[key][:] = f[key][:][SR_range]
                f_SB[key][:] = f[key][:][BR_range]


def split_SR_BR_deta(h5_path, output_path, deta_cut=3.1):

    # read data
    with h5py.File(h5_path, 'r') as f:
        mjj = f['EVENT/mjj'][:]
        deta = f['EVENT/deta'][:]

        SR_range = deta > deta_cut
        BR_range = deta < deta_cut
        
        root, _ = os.path.splitext(output_path)
        SR_path = f'{root}_in_SR.h5'
        BR_path = f'{root}_in_BR.h5'

        with h5py.File(SR_path, 'w') as f_SR, h5py.File(BR_path, 'w') as f_SB:

            create_dataset(f_SR, SR_range.sum(), MAX_CONSTI)
            create_dataset(f_SB, BR_range.sum(), MAX_CONSTI)

            keys = get_dataset_keys(f_SR)
            for key in keys:
                f_SR[key][:] = f[key][:][SR_range]
                f_SB[key][:] = f[key][:][BR_range]


def split_SR_BR_mjj_deta(h5_path, output_path, mjj_cut=300, deta_cut=3.1):

    # read data
    with h5py.File(h5_path, 'r') as f:
        mjj = f['EVENT/mjj'][:]
        deta = f['EVENT/deta'][:]

        SR_range = (mjj > mjj_cut) & (deta > deta_cut)
        BR_range = (mjj < mjj_cut) & (deta < deta_cut)
        
        root, _ = os.path.splitext(output_path)
        SR_path = f'{root}_in_SR.h5'
        BR_path = f'{root}_in_BR.h5'

        with h5py.File(SR_path, 'w') as f_SR, h5py.File(BR_path, 'w') as f_SB:

            create_dataset(f_SR, SR_range.sum(), MAX_CONSTI)
            create_dataset(f_SB, BR_range.sum(), MAX_CONSTI)

            keys = get_dataset_keys(f_SR)
            for key in keys:
                f_SR[key][:] = f[key][:][SR_range]
                f_SB[key][:] = f[key][:][BR_range]

In [4]:
h5_path = 'data/GGF-03.h5'
output_path = 'data/mjj_cut/GGF.h5'
split_SR_BR_mjj(h5_path, output_path)

h5_path = 'data/VBF-03.h5'
output_path = 'data/mjj_cut/VBF.h5'
split_SR_BR_mjj(h5_path, output_path)

# $\phi$ shifting and flipping

In [5]:
def to_event_image_h5(h5_path, out_h5):

    shutil.copyfile(h5_path, out_h5)

    with h5py.File(out_h5, 'a') as f_out:
        print(out_h5)

        event_pt = np.concatenate([f_out['TOWER/pt'][:], f_out['TRACK/pt'][:], f_out['PHOTON/pt'][:]], axis=1)
        event_eta = np.concatenate([f_out['TOWER/eta'][:], f_out['TRACK/eta'][:], f_out['PHOTON/eta'][:]], axis=1)
        event_phi = np.concatenate([f_out['TOWER/phi'][:], f_out['TRACK/phi'][:], f_out['PHOTON/phi'][:]], axis=1)

        _, _, new_phi = phi_shift_and_flipping(event_pt, event_eta, event_phi)

        f_out['TOWER/phi'][:] = new_phi[:, :MAX_CONSTI['Tower']]
        f_out['TRACK/phi'][:] = new_phi[:, MAX_CONSTI['Tower']:MAX_CONSTI['Tower']+MAX_CONSTI['Track']]
        f_out['PHOTON/phi'][:] = new_phi[:, MAX_CONSTI['Tower']+MAX_CONSTI['Track']:]

In [6]:
for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
    h5_path = f'./data/mjj_cut/{name}.h5'
    out_h5 = f'./data/mjj_cut/pre-processing/{name}.h5'
    to_event_image_h5(h5_path, out_h5)

./data/mjj_cut/pre-processing/VBF_in_SR.h5
./data/mjj_cut/pre-processing/VBF_in_BR.h5
./data/mjj_cut/pre-processing/GGF_in_SR.h5
./data/mjj_cut/pre-processing/GGF_in_BR.h5


# Pixelation

In [7]:
res = 40
h5_dir = './data/mjj_cut/pre-processing'
npy_dir = f'./data/mjj_cut/pre-processing/{res}x{res}'

# create output directory
if not os.path.exists(npy_dir):
    os.makedirs(npy_dir)

for name in ['VBF_in_SR', 'VBF_in_BR', 'GGF_in_SR', 'GGF_in_BR']:
    h5_path = os.path.join(h5_dir, f'{name}.h5')
    npy_path = os.path.join(npy_dir, f'{name}.npy')
    cmd = f'python pixelation.py {h5_path} {npy_path} {res} &'
    print(cmd)

python pixelation.py ./data/mjj_cut/pre-processing/VBF_in_SR.h5 ./data/mjj_cut/pre-processing/40x40/VBF_in_SR.npy 40 &
python pixelation.py ./data/mjj_cut/pre-processing/VBF_in_BR.h5 ./data/mjj_cut/pre-processing/40x40/VBF_in_BR.npy 40 &
python pixelation.py ./data/mjj_cut/pre-processing/GGF_in_SR.h5 ./data/mjj_cut/pre-processing/40x40/GGF_in_SR.npy 40 &
python pixelation.py ./data/mjj_cut/pre-processing/GGF_in_BR.h5 ./data/mjj_cut/pre-processing/40x40/GGF_in_BR.npy 40 &
